In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
from pathlib import Path
import itertools

In [ ]:
throughput = {}
pause_time = {}
out = {}
for gc in ["G1", "Parallel", "Z"]:
    for runtime in ["Graal", "HotSpot"]:
        files = list(Path("benchmark_stats/gc_stats").glob(f"{gc}_{runtime}*"))
        assert len(files) == 1
        file = files[0]
        with open(file) as f:
            stats = json.loads(f.read())["stats"]
            assert isinstance(stats, list)
            for stat in stats:
                heap_size = stat["heap_size"]
                throughput.setdefault(runtime, {}).setdefault(heap_size, {})["avg_throughput"] = stat["avg_throughput"]
                pause_time.setdefault(runtime, {}).setdefault(heap_size, {})["p90_avg_pause_time"] = stat["p90_avg_pause_time"]
    hot_t, graal_t = throughput["HotSpot"], throughput["Graal"]
    hot_p, graal_p = pause_time["HotSpot"], pause_time["Graal"]
    for heap_size in hot_t:
        t_hot = hot_t[heap_size]["avg_throughput"]
        t_graal = graal_t[heap_size]["avg_throughput"]
        p_hot = hot_p[heap_size]["p90_avg_pause_time"]
        p_graal = graal_p[heap_size]["p90_avg_pause_time"]
        out.setdefault(gc, {})[heap_size] = {
            "throughput_diff_ns": t_graal - t_hot,
            "throughput_percentage": (t_graal - t_hot)/t_hot * 100, 
            "pause_time_diff_ns": p_graal - p_hot, 
            "pause_time_percentage": (p_graal - p_hot)/p_hot  * 100, 
        }
print(json.dumps(out))

In [ ]:
for file in Path("graphs").glob("**/*.png"):
    file.unlink()

In [ ]:
dir = Path("benchmark_matrix")
with open(dir / "Graal_11_10_0_19.json") as f:
    graal = json.loads(f.read())["matrix"]
with open(dir / "HotSpot_11_10_0_19.json") as f:
    hotspot = json.loads(f.read())["matrix"]

In [ ]:
def plot_matrix(key, d, file_title, y_label, y_limit, layout=None):
    x_labels = list(d.keys())
    x = np.arange(len(x_labels))
    fig, ax = plt.subplots(figsize=(20, 10), layout=layout, dpi=300)
    
    gcs = sorted(graal[x_labels[0]])
    #def plot_sub_graph(ax, key, d, gcs, title, y_label, y_limit):
    width = 0.3
    multiplier = 0
    for gc in gcs:
        values = [value[gc][key] for value in d.values()]
        offset = width * multiplier
        rects = ax.bar(x + offset, values, width, label=gc)
        ax.bar_label(rects)
        multiplier += 1
        ## Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel(y_label)
        ax.set_xlabel("Heap Sizes (MB)")
        #ax.set_title(title)
        ax.set_xticks(x + width, x_labels)
        ax.legend(ncols=3, fontsize=20)
        ax.set_ylim(0, y_limit)
        fig.savefig(f"graphs/{file_title}.png")
            

In [ ]:
plot_matrix("throughput", graal,  "throughput_graalvm", "Throughput", 2, "tight")
plot_matrix("throughput", hotspot, "throughput_hotspot", "Throughput", 2, "tight")
plot_matrix("pause_time", graal, "pause_time_graalvm", "Pause Time", 2, "tight")
plot_matrix("pause_time", hotspot,  "pause_time_hotspot", "Pause Time",2.6, "tight" )

In [ ]:
def load_benchmarks(jdk, benchmark_group):
    benchmarks = {}
    files = [f for f in Path(".").glob("benchmark_stats/*.json") if jdk in f.name and benchmark_group in f.name and "error" not in f.name]
    for file in files:
        with open(file) as f:
            benchmark = json.loads(f.read())
            heap_size = benchmark["heap_size"]
            name = benchmark["benchmark_name"]
            gc = benchmark["garbage_collector"]
            throughput =  benchmark["throughput"]
            pause_time =  benchmark["p90_pause_time"]
            a = benchmarks.setdefault(heap_size, {}).setdefault(name, {}).setdefault(gc, {})
            a["throughput"] = throughput / 1e9
            a["pause_time"] = pause_time / 1e9
            
    # filter results not valid for all 3 GCs
    #print("Before", str(benchmarks)[:500])
    for key, value in list(benchmarks.items()):
        for k, v in list(value.items()):
           #print(f"Key={k} Value={v}")
           if len(v) != 3 or k == "spring": # filter spring
               #print(f"Deleting with keys: {key},{k}: {benchmarks[key][k]}")
               del benchmarks[key][k]
               #print(f"After delete with keys: {key},{k}: {benchmarks[key].get(k)}")
           #else:
           #    benchmarks[key] = dict(sorted(value.items()))
    for key in list(benchmarks):
        #print(f"Before: {benchmarks[key]}")
        benchmarks[key] = dict(sorted(benchmarks[key].items()))
        #print(f"After: {benchmarks[key]}")

    # assert everything is sorted
    # heap_size : name: gc
    for key in benchmarks:
        assert list(benchmarks[key].keys()) == sorted(list(benchmarks[key].keys())), f"Error: {list(benchmarks[key].keys())=} != {sorted(list(benchmarks[key].keys()))=}"
        #print(benchmarks[key].keys())
        for name in benchmarks[key]:
            assert len(benchmarks[key][name]) == 3, f"Error, {benchmarks[key][name]}"
        #a = [f"{benchmarks[key]} {v=} {len(v)=}" for v in benchmarks[key].values()]
        #print(a)
    #assert len(benchmarks['256']['kafka']) == 3, "hmm"
    #print(benchmarks['256']['kafka'])
    #print(benchmarks.keys())
    
        
    return benchmarks

In [ ]:

a = load_benchmarks("HotSpot", "DaCapo")
#for gc in ["Parallel", "G1", "Z"]:
#for key1 in a:
#    assert len(a[key1]) == 3

#print(a)
#print(a['512'].keys())
#print(a['512'].keys())
#print(a['256']['tradesoap']['Parallel'])

In [ ]:
def plot_benchmarks(benchmarks, key, y_label, file_title, layout=None, y_limit=None):
    x_labels = list(benchmarks.keys()) # This should be already sorted
    print(f"{x_labels=}")
    x = np.arange(len(x_labels))
    fig, ax = plt.subplots(figsize=(20, 10), layout=layout, dpi=300)

    gcs = sorted(benchmarks[x_labels[0]])
    width = 0.3
    multiplier = 0
    for gc in gcs:
        values = [value[gc][key] for value in benchmarks.values()]
        offset = width * multiplier
        rects = ax.bar(x + offset, values, width, label=gc)
        multiplier += 1
        ax.set_ylabel(y_label)
        ax.set_xlabel("Benchmarks")
        ax.set_xticks(x + width, x_labels)
        plt.xticks(rotation=45)
        ax.legend(ncols=3)
        ax.set_ylim(0, y_limit)
        #ax.set_title(file_title)
        #ax.bar_label(rects, label_type='center')
        fig.savefig(f"graphs/{file_title}.png")
        
def find_increment(number):
    n = str(number).split(".")
    num = 1
    if n[0] == "0":
        num = 0.1
        for i in n[1]:
            num *= 0.1
            if i != "0":
                break
    return num

    
#plt.tight_layout()
plt.rcParams.update({'font.size': 22})
for runtime, group in itertools.product(["Graal", "HotSpot"], ["Renaissance", "DaCapo"]):
    benchmarks = load_benchmarks(runtime, group)
    for heap_size in benchmarks:
        plt.close("all")
        max_throughput = max([v2["throughput"] for v1 in benchmarks[heap_size].values() for v2 in v1.values()])
        max_latency = max([v2["pause_time"] for v1 in benchmarks[heap_size].values() for v2 in v1.values()])
        #print("Antes", max_throughput, max_latency)
        max_latency += find_increment(max_latency)
        max_throughput += find_increment(max_throughput)
        #print("Depois", max_throughput, max_latency)
        plot_benchmarks(benchmarks[heap_size], "throughput", "Throughput (seconds)", f"{runtime}/{group}/{runtime}-{group}-{heap_size}-throughput", layout='tight',y_limit=max_throughput)
        plot_benchmarks(benchmarks[heap_size], "pause_time", "Pause Time (seconds)", f"{runtime}/{group}/{runtime}-{group}-{heap_size}-pause_time", layout='tight',y_limit=max_latency)

In [37]:
def plot_benchmarks(benchmarks, y_label, file_title, layout=None, y_limit=None):
    # benchmarks -> dict[heap_size, dict[gc, value]]
    x_labels = list(benchmarks) # This should be already sorted
    print(f"{x_labels=}") # x_lables = [heap_size]
    x = np.arange(len(x_labels))
    fig, ax = plt.subplots(figsize=(20, 10), layout=layout, dpi=300)

    gcs = sorted(benchmarks[x_labels[0]]) # sorted gcs
    width = 0.3
    multiplier = 0
    for gc in gcs:
        values = [d[gc] for d in benchmarks.values()]
        offset = width * multiplier
        rects = ax.bar(x + offset, values, width, label=gc)
        multiplier += 1
        ax.set_ylabel(y_label)
        ax.set_xlabel("Heap Sizes (MB)")
        ax.set_xticks(x + width, x_labels)
        #plt.xticks(rotation=45)
        ax.legend(ncols=3)
        ax.set_ylim(0, y_limit)
        #ax.set_title(file_title)
        #ax.bar_label(rects, label_type='center')
        fig.savefig(f"graphs/{file_title}.png")
        
def find_increment(number):
    n = str(number).split(".")
    num = 1
    if n[0] == "0":
        num = 0.1
        for i in n[1]:
            num *= 0.1
            if i != "0":
                break
    return num

    
#plt.tight_layout()
plt.rcParams.update({'font.size': 22})
data = {}
for runtime in ["Graal", "HotSpot"]:
    benchmarks = list(Path("spring_stats").glob(f"*{runtime}*"))
    for benchmark in benchmarks:
        with open(benchmark) as f:
            d = json.loads(f.read())
            assert runtime in d["jdk"]
            assert "spring" == d["benchmark_name"]
            assert d["error"] is None
            heap_size = d["heap_size"]
            gc = d["garbage_collector"]
            p90 = d["p90_pause_time"]
            throughput = d["throughput"]
            data.setdefault(runtime, {}).setdefault("throughput", {}).setdefault(int(heap_size), {})[gc] = throughput / 1e9
            data.setdefault(runtime, {}).setdefault("pause_time", {}).setdefault(int(heap_size), {})[gc] = p90 / 1e9

for key in list(data):
    data[key]["throughput"] = dict(sorted(data[key]["throughput"].items()))
    data[key]["pause_time"] = dict(sorted(data[key]["pause_time"].items()))
    assert list(data[key]["throughput"]) == sorted(data[key]["throughput"])
    assert list(data[key]["pause_time"]) == sorted(data[key]["pause_time"])

print(json.dumps(data))
return
for runtime in data:
    throughput_data = data[runtime]["throughput"]
    pause_data = data[runtime]["pause_time"]
    plt.close("all")
    max_throughput = max([v2 for gcs_dict in throughput_data.values() for v2 in gcs_dict.values()])
    max_latency = max([v2 for gcs_dict in pause_data.values() for v2 in gcs_dict.values()])
    #print("Antes", max_throughput, max_latency)
    max_latency += find_increment(max_latency)
    max_throughput += find_increment(max_throughput)
    #print("Depois", max_throughput, max_latency)
    plot_benchmarks(throughput_data, "Throughput (seconds)", f"BestGC_plusplus/{runtime}/BestGC_{runtime}-throughput", layout='tight',y_limit=max_throughput)
    plot_benchmarks(pause_data, "Pause Time (seconds)", f"BestGC_plusplus/{runtime}/BestGC_{runtime}-pause_time", layout='tight',y_limit=max_latency)

{"Graal": {"throughput": {"256": {"G1": 100.956143022, "Z": 172.133560401, "Parallel": 100.721224507}, "512": {"Z": 111.566554946, "Parallel": 95.221145106, "G1": 97.730748951}, "1024": {"Parallel": 94.448982644, "G1": 96.986427753, "Z": 100.82124546}, "2048": {"Parallel": 92.472127809, "G1": 96.924042931, "Z": 96.371266886}, "4096": {"Parallel": 91.557172331, "G1": 96.931326247, "Z": 96.587391626}, "8192": {"G1": 96.032675242, "Z": 97.710785503, "Parallel": 91.880692109}}, "pause_time": {"256": {"G1": 0.011665659, "Z": 0.0020112408, "Parallel": 0.0084089566}, "512": {"Z": 0.0030161742, "Parallel": 0.008539155199999999, "G1": 0.01274278}, "1024": {"Parallel": 0.0086615358, "G1": 0.014064722699999999, "Z": 0.003198721}, "2048": {"Parallel": 0.0079225928, "G1": 0.014682675, "Z": 0.0023168136000000002}, "4096": {"Parallel": 0.0074958221, "G1": 0.020586179, "Z": 0.0021158837999999997}, "8192": {"G1": 0.025045479, "Z": 0.0021154329, "Parallel": 0.0064246653}}}, "HotSpot": {"throughput": {"2

SyntaxError: 'return' outside function (1784715630.py, line 63)

In [ ]:
def find_only_success():
    benchmarks = {}
    for file in Path("benchmark_stats/").glob("Renaissance*256*Graal*.json"):
        #print(file)
        with open(file) as f: 
            benchmark = json.loads(f.read())
        key = "error" if "error" in f"{file}" else "success"
        assert benchmark["benchmark_name"] is not None
        benchmarks.setdefault(benchmark["benchmark_name"], {}).setdefault(key, []).append(benchmark["garbage_collector"])
    import pprint
    for key in benchmarks:
        if len(benchmarks[key].get("success", {})) == 1:
            print(key)
find_only_success()

In [ ]:
def compute_benchmarks_len_report():
    benchmarks = {}
    for r in ["Graal", "HotSpot"]:
        for group in ["DaCapo", "Renaissance"]:
            for file in Path("benchmark_stats/").glob(f"{group}*{r}*.json"):
                with open(file) as f: 
                    b = json.loads(f.read())
                    gc = b["garbage_collector"]
                    name = b["benchmark_name"]
                    if name == "spring":
                        print("Skipping")
                        continue
                    status = "error" if bool(b["error"]) else "success"
                    hs = b["heap_size"]
                    entry = benchmarks.setdefault(r, {}).setdefault(hs, {}).setdefault(gc, {}).setdefault(status, [])
                    entry.append(name)
    nums = {}
    for r, h_dict in benchmarks.items():
        for heap, gc_dict in h_dict.items():
            for gc in gc_dict:
                error = gc_dict[gc].get("error", [])
                success = gc_dict[gc].get("success", [])
                error = len(error)
                success = len(success)
                gc_dict[gc]["error"] = error
                gc_dict[gc]["success"] = success
                nums.setdefault(heap,[]).append(error + success)
                
    for key in nums:
        print(f"{nums[key]=}")
        assert len(set(nums[key])) == 1
    
    with open("benchmarks_len_report.json", "w") as f:
        f.write(json.dumps(benchmarks))
    print(benchmarks)
compute_benchmarks_len_report()
    

In [38]:
def eq1(t_weight, t_score, p_weight, p_score):
    return t_weight * t_score + p_weight * p_score
    
def eq2(t_weight, t_score, p_weight, p_score):
    if t_weight == 0:
        t_weight = t_score = 1
    if p_weight == 0:
        p_weight = p_score = 1
    return t_weight * t_score * p_weight * p_score

def all():
    dir = Path("benchmark_matrix")
    with open(dir / "Graal_11_10_0_19.json") as f:
        graal = json.loads(f.read())["matrix"]["512"]
    with open(dir / "HotSpot_11_10_0_19.json") as f:
        hotspot = json.loads(f.read())["matrix"]["512"]
    
    for (t,p) in zip([1,0],[0,1]):
        for gc in graal:
            gc_t = graal[gc]["throughput"]
            gc_p = graal[gc]["pause_time"]
            #print(gc_t, gc_p)
            score_1 = eq1(t, gc_t, p, gc_p)
            score_2 = eq2(t, gc_t, p, gc_p)
            print(f"Graal: {gc}, {t=}, {p=} {score_1=} {score_2=}")
    for (t,p) in zip([1,0],[0,1]):
        for gc in hotspot:
            gc_t = graal[gc]["throughput"]
            gc_p = graal[gc]["pause_time"]
            score_1 = eq1(t, hotspot[gc]["throughput"], p, hotspot[gc]["pause_time"])
            score_2 = eq2(t, hotspot[gc]["throughput"], p, hotspot[gc]["pause_time"])
            print(f"HotSpot: {gc}, {t=}, {p=} {score_1=} {score_2=}")


t_weight = 0.72
p_weight = 0.28
old = eq1(t_weight, 95, p_weight, 0.008539)
new = eq2(t_weight, 95, p_weight, 0.008539)
print(f"Graal: Parallel: {old=} {new=}")
old = eq1(t_weight, 112, p_weight, 0.003016)
new = eq2(t_weight, 112, p_weight, 0.003016)
print(f"Graal: Z: {old=} {new=}")
old = eq1(t_weight, 98, p_weight, 0.012743)
new = eq2(t_weight, 98, p_weight, 0.012743)
print(f"Graal: G1: {old=} {new=}")


t_weight = 0.71
p_weight = 0.29
old = eq1(t_weight, 91, p_weight, 0.008807)
new = eq2(t_weight, 91, p_weight, 0.008807)
print(f"HotSpot: Parallel: {old=} {new=}")
old = eq1(t_weight, 107, p_weight, 0.003524)
new = eq2(t_weight, 107, p_weight, 0.003524)
print(f"HotSpot: Z: {old=} {new=}")
old = eq1(t_weight, 92, p_weight, 0.012875)
new = eq2(t_weight, 92, p_weight, 0.012875)
print(f"HotSpot: G1: {old=} {new=}")



Graal: Parallel: old=68.40239091999999 new=0.163538928
Graal: Z: old=80.64084448 new=0.06809886720000001
Graal: G1: old=70.56356804 new=0.25176090240000004
HotSpot: Parallel: old=64.61255403 new=0.1650158783
HotSpot: Z: old=75.97102196 new=0.0776383012
HotSpot: G1: old=65.32373374999999 new=0.24388854999999995


In [ ]:
def eq1(t_weight, t_score, p_weight, p_score):
    return t_weight * t_score + p_weight * p_score
    
def eq2(t_weight, t_score, p_weight, p_score):
    if t_weight == 0:
        t_weight = t_score = 1
    if p_weight == 0:
        p_weight = p_score = 1
    return t_weight * t_score * p_weight * p_score
